In [1]:
from pygod.utils import load_data
from typing import List
from gad_adversarial_robustness.gad.dominant.dominant import Dominant
from gad_adversarial_robustness.gad.dominant.dominant_lip import DominantLIP
from gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset, get_n_anomaly_indexes
import argparse
import torch
from torch_geometric.utils import to_dense_adj
import numpy as np
from gad_adversarial_robustness.poison.greedy import multiple_AS, poison_attack
import copy

In [2]:
data = load_data("inj_cora")
truth: List[int] = data.y.bool()

adj, attrs, label, adj_label = load_anomaly_detection_dataset(data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)

class Args:
    def __init__(self, hidden_dim=64, epoch=100, lr=5e-3, dropout=0.3, alpha=0.8, device='cpu'):
        self.hidden_dim = hidden_dim
        self.epoch = epoch
        self.lr = lr
        self.dropout = dropout
        self.alpha = alpha
        self.device = device

# Create an instance of the Args class with default values
args = Args()


dominant = Dominant(feat_size = attrs.size(1), hidden_size = args.hidden_dim, dropout = args.dropout, device = args.device, adj=adj, adj_label=adj_label, attrs=attrs, label=label)
dominant.fit(args)


Epoch: 0000 Auc 0.8398832684824901
Epoch: 0010 Auc 0.8399001860937236
Epoch: 0020 Auc 0.8397845824169627
Epoch: 0030 Auc 0.8397366491851351
Epoch: 0040 Auc 0.839787402018835
Epoch: 0050 Auc 0.8397930412225793
Epoch: 0060 Auc 0.8398240568431736
Epoch: 0070 Auc 0.8397563863982406
Epoch: 0080 Auc 0.8398691704731291
Epoch: 0090 Auc 0.8398635312693847
Epoch: 0099 Auc 0.8399030056955958


In [3]:
import numpy as np
from gad_adversarial_robustness.utils.graph_utils import prepare_graph
dense_adj = dominant.adj
amount_of_nodes = data.num_nodes
_, adj, _ = prepare_graph(data)
dense_adj = adj.to_dense()  #Fill in zeroes where there are no edges

print("Create poison compatible adjacency matrix...")
triple = []
for i in range(amount_of_nodes):
    for j in range(i + 1, amount_of_nodes):
        triple.append([i, j, dense_adj[i,j]])  #Fill with 0, then insert actual after



triple = np.array(triple)

#print(type(triple))
#triple = np.array([t.detach().numpy() for t in triple])
#np.array(triple)




# These are the nodes we try reduce the active subnetwork score for (disguising anonomaly nodes)
target_node_lst = get_n_anomaly_indexes(truth, 999)
#target_node_lst = get_n_anomaly_indexes(truth, 10)
print(type(target_node_lst)), print(f'target node list: {target_node_lst}'), print(target_node_lst)

print("Making model...")
model = multiple_AS(target_lst = target_node_lst, n_node = amount_of_nodes, device = 'cpu')
budget = 2  # The amount of edges to change



tensor([1., 1., 1.,  ..., 1., 1., 1.])
node_attrs
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
adj
SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             size=(2708, 2708), nnz=11054, density=0.15%)
labels
tensor([0, 0, 0,  ..., 0, 0, 0])
Create poison compatible adjacency matrix...
Anomalies indexes: [  10   50   70   76  104  124  127  143  151  170  179  181  196  214
  217  224  227  287  289  294  311  333  425  438  451  454  459  539
  565  572  578  581  615  619  641  652  654  660  670  674  692  711
  722  738  781  833  843  869  874  878  882  891  895  915  923  938
  980  982  996 1002 1014 1035 1053 1079 1090 1096 1133 1135 1206 1211
 1224 1229 1235 1287 1293 1310 1362 1391 1414 1426 1533 1540 1543 1547
 1570 1

In [4]:
# shape of triple
print(f'shape of triple: {len(triple)}')
print(type(triple))
print(type(triple[2][1]))
print("Starting attack...")
#triple = triple.detach().cpu().numpy()

#triple = torch.tensor(triple, requires_grad=True)


adj_adversary, _, _ = poison_attack(model, triple, budget)

print("Converting to compatible tensor...")

# Create Edge Index'
edge_index = torch.tensor([[],[]])

# Transpose it to make shape compatible
transposed_adj_adversary = torch.transpose(adj_adversary, 0, 1)

for i in range(len(adj_adversary)):
    if(adj_adversary[i][2] != 0):   #If edge value is not 0 (no edge)
        #Add edge to edge index, choosing first 2 elements (edges), and then the ith edge
        edge_index = torch.cat((edge_index, transposed_adj_adversary[:2, i:i+1]), -1)
        # Dataset uses edges both ways so add reverse edge as well
        edge_index = torch.cat((edge_index, torch.flip(transposed_adj_adversary[:2, i:i+1], dims=[0])), -1)



# Make new data object with new edge index
data_after_poison = copy.deepcopy(data)
edge_index = edge_index.type(torch.int64)
data_after_poison.edge_index = edge_index








shape of triple: 3665278
<class 'numpy.ndarray'>
<class 'numpy.float64'>
Starting attack...
triple copy type: <class 'numpy.ndarray'>


c:\users\andwh\documents\p6\aau-dat6-poison-gad\gad_adversarial_robustness\poison\greedy.py:35: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return torch.sparse.mm(torch.sparse.mm(A_sp, A_sp), A_sp).to_dense()
c:\users\andwh\documents\p6\aau-dat6-poison-gad\gad_adversarial_robustness\poison\greedy.py:39: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3618.)
  E = torch.sum(A, 1) + 0.5 * torch.diag(self.spars

Converting to compatible tensor...


In [5]:
"""
def compare_edge_indexes(edge_index1, edge_index2):
    # Convert edge_index tensors to NumPy arrays
    edge_index1_np = edge_index1.numpy()
    edge_index2_np = edge_index2.numpy()

    # Sort the arrays along the second dimension (columns)
    edge_index1_sorted = edge_index1_np[:, edge_index1_np[1].argsort()]
    edge_index2_sorted = edge_index2_np[:, edge_index2_np[1].argsort()]

    # Get the minimum number of edges between the two tensors
    min_edges = min(edge_index1_sorted.shape[1], edge_index2_sorted.shape[1])

    # Compare the sorted arrays element-wise for the common edges
    modified_edges = []
    for i in range(min_edges):
        if not np.array_equal(edge_index1_sorted[:, i], edge_index2_sorted[:, i]):
            modified_edges.append((edge_index1_sorted[0, i], edge_index1_sorted[1, i]))

    # Check for additional edges in edge_index1
    if edge_index1_sorted.shape[1] > min_edges:
        for i in range(min_edges, edge_index1_sorted.shape[1]):
            modified_edges.append((edge_index1_sorted[0, i], edge_index1_sorted[1, i]))

    # Check for additional edges in edge_index2
    if edge_index2_sorted.shape[1] > min_edges:
        for i in range(min_edges, edge_index2_sorted.shape[1]):
            modified_edges.append((edge_index2_sorted[0, i], edge_index2_sorted[1, i]))

    return modified_edges


print("Modified edges:")

modified_edges = compare_edge_indexes(data.edge_index, data_after_poison.edge_index)
for edge in modified_edges:
    print(edge)
"""

'\ndef compare_edge_indexes(edge_index1, edge_index2):\n    # Convert edge_index tensors to NumPy arrays\n    edge_index1_np = edge_index1.numpy()\n    edge_index2_np = edge_index2.numpy()\n\n    # Sort the arrays along the second dimension (columns)\n    edge_index1_sorted = edge_index1_np[:, edge_index1_np[1].argsort()]\n    edge_index2_sorted = edge_index2_np[:, edge_index2_np[1].argsort()]\n\n    # Get the minimum number of edges between the two tensors\n    min_edges = min(edge_index1_sorted.shape[1], edge_index2_sorted.shape[1])\n\n    # Compare the sorted arrays element-wise for the common edges\n    modified_edges = []\n    for i in range(min_edges):\n        if not np.array_equal(edge_index1_sorted[:, i], edge_index2_sorted[:, i]):\n            modified_edges.append((edge_index1_sorted[0, i], edge_index1_sorted[1, i]))\n\n    # Check for additional edges in edge_index1\n    if edge_index1_sorted.shape[1] > min_edges:\n        for i in range(min_edges, edge_index1_sorted.shape[

In [6]:
from gad_adversarial_robustness.poison.greedy import inject_greedy_poison
poison_adj, poison_attrs, poison_label, poison_adj_label = load_anomaly_detection_dataset(data_after_poison)
poison_adj = torch.FloatTensor(poison_adj)
poison_adj_label = torch.FloatTensor(poison_adj_label)
poison_attrs = torch.FloatTensor(poison_attrs)
tuple = (poison_attrs, [1, 2, 3])
print(tuple[0][1])

class Args:
    def __init__(self, hidden_dim=64, epoch=100, lr=5e-3, dropout=0.3, alpha=0.8, device='cpu', lip_weight = 1.5):
        self.hidden_dim = hidden_dim
        self.epoch = epoch
        self.lr = lr
        self.dropout = dropout
        self.alpha = alpha
        self.device = device
        self.lip_weight = lip_weight

# Create an instance of the Args class with default values
args = Args()

print("DOMINANT ACCURACY: ")
dominant = Dominant(feat_size = attrs.size(1), hidden_size = args.hidden_dim, dropout = args.dropout, device = args.device, adj=poison_adj, adj_label=poison_adj_label, attrs=poison_attrs, label=poison_label)
dominant.fit(args)

print("DOMINANT_LIP ACCURACY: ")
dominant_lip = DominantLIP(feat_size = attrs.size(1), hidden_size = args.hidden_dim, dropout = args.dropout, device = args.device, adj=poison_adj, adj_label=poison_adj_label, attrs=poison_attrs, label=poison_label)
dominant_lip.fit(0.001, args)



"""
pred, score, prob, conf = dominant.predict(data_after_poison,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)

pred_lip, score_lip, prob_lip, conf_lip = dominant_lip.predict(data_after_poison,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)


print(f'prediction: {pred}')
print(f'score: {score}')
print(f'probability: {prob}')
print(f'confidence: {conf}')
print()
print("DominantLIP Model:")
print(f'prediction: {pred_lip}')
print(f'score: {score_lip}')
print(f'probability: {prob_lip}')
print(f'confidence: {conf_lip}')
print()

"""





tensor([0., 0., 0.,  ..., 0., 0., 0.])
DOMINANT ACCURACY: 
Epoch: 0000 Auc 0.8258783059831952
Epoch: 0010 Auc 0.8381604917385665
Epoch: 0020 Auc 0.8380505272655501
Epoch: 0030 Auc 0.8380730840805279
Epoch: 0040 Auc 0.8382112445722665
Epoch: 0050 Auc 0.8382464895956693
Epoch: 0060 Auc 0.8382253425816275
Epoch: 0070 Auc 0.8382760954153273
Epoch: 0080 Auc 0.8382619974059664
Epoch: 0090 Auc 0.8383381266565161
Epoch: 0099 Auc 0.8384368127220436
DOMINANT_LIP ACCURACY: 


: 